In [0]:
import urllib

In [0]:
Access_key = "****"
Secret_key = "****"
Encoded_secret_key = urllib.parse.quote(Secret_key, "")
Bucket_name = "my-cluster87235"
Mount_name = "mount1"
dbutils.fs.mount("s3a://%s:%s@%s" %(Access_key, Encoded_secret_key, Bucket_name), "/mnt/%s" % Mount_name)

In [0]:
dbutils.fs.ls("/mnt/mount1")

In [0]:
dbutils.fs.head("/mnt/mount1/listings.csv")

In [0]:
dbutils.fs.help()

In [0]:
File_loc = "dbfs:/mnt/mount1/listings.csv"
File_type = "csv"

row_header = "true"
delimeter = ","
infer_Schema = "true"

df1 = spark.read.format(File_type) \
    .option("header", row_header) \
    .option("inferSchema", infer_Schema) \
    .option("delimeter", delimeter) \
    .load(File_loc)

In [0]:
display(df1)

In [0]:
df1.printSchema()

In [0]:
df1.tail(1000)

In [0]:
df1.select("id", "name", "price").show()

In [0]:
from pyspark.sql.functions import *

In [0]:
df2 = df1.withColumn("price", col("price").cast("int"))

In [0]:
df2.printSchema()

In [0]:
display(df1.select("id", "name", "price"))

In [0]:
df2.write.mode("overwrite").format("parquet").option("path", "/user/external_tables/").saveAsTable("listing2")


In [0]:
df2.write.mode("overwrite").format("parquet").saveAsTable("listing")

In [0]:
%sql
select * from listing

In [0]:
%sql
select * from listing2

In [0]:
%sql

select id, name, host_id, host_name, price from listing2
where price > (select avg(price) from listing2)

In [0]:
%sql

select id, name, host_id, host_name, price from listing2
where price > 1000

In [0]:
df_price = df1.where(col("price") > 1000)

In [0]:
df_price2 = df_price.select("id","name","host_id","host_name", "price").show()

In [0]:
df_price_min_nights = df.where((col("price") > 2500) & (col("minimum_nights") > "5")) \
                       .select("id","name","host_id","host_name", "price", "minimum_nights").show()

In [0]:
from pyspark.sql.functions import *

In [0]:
df_price_min_nights.repartition(1).write.csv("dbfs:/mnt/mount1/list_price_min_ngt_data")

In [0]:
df_price2.repartition(1).write.csv("dbfs:/mnt/mount1/listing_fixed")

In [0]:
dbutils.fs.head("dbfs:/mnt/mount1/listing_fixed/")

In [0]:
dbutils.fs.unmount("/mnt/mount1")

In [0]:
-------------------------------------------------Thank You---------------------------------------